In [1]:
import pandas as pd

# Function to calculate F1 score
def calculate_f1(reference, hypothesis):
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()

    # Overlap tokens
    common_tokens = set(reference_tokens) & set(hypothesis_tokens)

    precision = len(common_tokens) / len(hypothesis_tokens) if len(hypothesis_tokens) > 0 else 0
    recall = len(common_tokens) / len(reference_tokens) if len(reference_tokens) > 0 else 0

    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1_score

# Load your CSVs
bengali_pdf = pd.read_csv('/content/mistral_bengali_answers (1).csv')
hindi_pdf   = pd.read_csv('/content/mistral_hindi_answers (1).csv')
kannada_pdf = pd.read_csv('/content/mistral_kannada_answers (1).csv')
marathi_pdf = pd.read_csv('/content/mistral_marathi_answers (1).csv')
# tamil_pdf   = pd.read_csv('/content/tamil_answers.csv')

# Store them in a dictionary for iteration
dataframes = {
    'bengali': bengali_pdf,
    'hindi': hindi_pdf,
    'kannada': kannada_pdf,
    'marathi': marathi_pdf
    # 'tamil': tamil_pdf
}

# Apply F1 calculation for each dataframe
for name, current_df in dataframes.items():
    current_df['f1_score'] = current_df.apply(
        lambda row: calculate_f1(str(row['answer_text']), str(row['generated_answer'])), axis=1
    )
    print(f"Overall F1 score for {name}: {current_df['f1_score'].mean()}")


Overall F1 score for bengali: 0.2829188656331296
Overall F1 score for hindi: 0.3538662085803525
Overall F1 score for kannada: 0.34975538220714986
Overall F1 score for marathi: 0.2632166516665437


In [2]:
%pip install evaluate
%pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0fe0f86cb9aaa306e3bd382b554f536ecaa8992364d8cfcbdd10029ed85f00d4
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [8]:
import pandas as pd
from evaluate import load

# F1 function
def calculate_f1(reference, hypothesis):
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()
    common_tokens = set(reference_tokens) & set(hypothesis_tokens)
    precision = len(common_tokens) / len(hypothesis_tokens) if hypothesis_tokens else 0
    recall = len(common_tokens) / len(reference_tokens) if reference_tokens else 0
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

# Load ROUGE metric
rouge_metric = load("rouge")

def calculate_rouge_scores(reference, hypothesis):
    if not reference.strip() or not hypothesis.strip():
        return {"rouge1": 0, "rouge2": 0, "rougeL": 0, "rougeLsum": 0}
    return rouge_metric.compute(predictions=[hypothesis], references=[reference])

# combined_df must already exist with 'answer_text' and 'generated_answer'
combined_df['f1_score'] = combined_df.apply(
    lambda row: calculate_f1(str(row['answer_text']), str(row['generated_answer'])), axis=1
)

# Compute ROUGE row-wise and convert to proper DataFrame
rouge_results = combined_df.apply(
    lambda row: calculate_rouge_scores(str(row['answer_text']), str(row['generated_answer'])), axis=1
)
rouge_df = pd.DataFrame(list(rouge_results))  # ✅ fix

# Merge with main df
combined_df = pd.concat([combined_df, rouge_df], axis=1)

# Print overall averages
print("Overall average F1 score:", combined_df['f1_score'].mean())

print("\nOverall average ROUGE scores:")
for col in rouge_df.columns:
    print(f"  {col}: {combined_df[col].mean()}")


Overall average F1 score: 0.31243927702179386

Overall average ROUGE scores:
  rouge1: rouge1    0.138718
rouge1    0.138718
rouge1    0.138718
dtype: float64
  rouge2: rouge2    0.060888
rouge2    0.060888
rouge2    0.060888
dtype: float64
  rougeL: rougeL    0.138718
rougeL    0.138718
rougeL    0.138718
dtype: float64
  rougeLsum: rougeLsum    0.138718
rougeLsum    0.138718
rougeLsum    0.138718
dtype: float64


In [4]:
%pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
from nltk.translate.meteor_score import meteor_score
from nltk.corpus import wordnet

def calculate_meteor(reference, hypothesis):
    reference_tokens = nltk.word_tokenize(reference)
    hypothesis_tokens = nltk.word_tokenize(hypothesis)
    # METEOR score requires a list of reference texts
    return meteor_score([reference_tokens], hypothesis_tokens)

combined_df['meteor_score'] = combined_df.apply(lambda row: calculate_meteor(row['answer_text'], row['generated_answer']), axis=1)

overall_meteor_score = combined_df['meteor_score'].mean()

print(f"Overall METEOR score for all files combined: {overall_meteor_score}")

Overall METEOR score for all files combined: 0.41163174037000255


In [6]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure punkt is downloaded if not already
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')
except LookupError:
    nltk.download('punkt')

def calculate_bleu(reference, hypothesis):
    reference_tokens = nltk.word_tokenize(reference)
    hypothesis_tokens = nltk.word_tokenize(hypothesis)
    # Calculate BLEU-1 score (unigram precision)
    weights = (1, 0, 0, 0)
    return sentence_bleu([reference_tokens], hypothesis_tokens, weights=weights, smoothing_function=SmoothingFunction().method1)

combined_df['bleu1_score'] = combined_df.apply(lambda row: calculate_bleu(row['answer_text'], row['generated_answer']), axis=1)

overall_bleu1_score = combined_df['bleu1_score'].mean()

print(f"Overall BLEU-1 score for all files combined: {overall_bleu1_score}")

Overall BLEU-1 score for all files combined: 0.22329169503073326


In [10]:
import pandas as pd
from evaluate import load
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk

# Ensure punkt is downloaded if not already
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')
except LookupError:
    nltk.download('punkt')


# --------------------------
# Load your CSVs (example)
# --------------------------
# Replace with your actual file paths
bengali_pdf = pd.read_csv('/content/mistral_bengali_answers (1).csv')
hindi_pdf   = pd.read_csv('/content/mistral_hindi_answers (1).csv')
kannada_pdf = pd.read_csv('/content/mistral_kannada_answers (1).csv')
marathi_pdf = pd.read_csv('/content/mistral_marathi_answers (1).csv')

# Combine into one dataframe
combined_df = pd.concat([bengali_pdf, hindi_pdf, kannada_pdf, marathi_pdf],
                        axis=0, ignore_index=True)

# --------------------------
# F1 Score
# --------------------------
def calculate_f1(reference, hypothesis):
    reference_tokens = str(reference).split()
    hypothesis_tokens = str(hypothesis).split()
    common_tokens = set(reference_tokens) & set(hypothesis_tokens)
    precision = len(common_tokens) / len(hypothesis_tokens) if hypothesis_tokens else 0
    recall = len(common_tokens) / len(reference_tokens) if reference_tokens else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

combined_df['f1_score'] = combined_df.apply(
    lambda row: calculate_f1(str(row['answer_text']), str(row['generated_answer'])), axis=1
)

# --------------------------
# ROUGE
# --------------------------
rouge_metric = load("rouge")

def calculate_rouge_scores(reference, hypothesis):
    reference = str(reference).strip()
    hypothesis = str(hypothesis).strip()
    if not reference or not hypothesis:
        return {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0, "rougeLsum": 0.0}
    results = rouge_metric.compute(predictions=[hypothesis], references=[reference])
    # Extract scalar values
    return {k: float(v) for k, v in results.items()}

# Compute ROUGE row-wise
rouge_results = combined_df.apply(
    lambda row: calculate_rouge_scores(row['answer_text'], row['generated_answer']), axis=1
)
rouge_df = pd.DataFrame(list(rouge_results))
combined_df = pd.concat([combined_df, rouge_df], axis=1)

# --------------------------
# METEOR
# --------------------------
def calculate_meteor(reference, hypothesis):
    reference = str(reference)
    hypothesis = str(hypothesis)
    if not reference.strip() or not hypothesis.strip():
        return 0.0
    reference_tokens = nltk.word_tokenize(reference)
    hypothesis_tokens = nltk.word_tokenize(hypothesis)
    return meteor_score([reference_tokens], hypothesis_tokens)

combined_df['meteor_score'] = combined_df.apply(
    lambda row: calculate_meteor(row['answer_text'], row['generated_answer']), axis=1
)

# --------------------------
# BLEU-1
# --------------------------
smooth_fn = SmoothingFunction().method1

def calculate_bleu1(reference, hypothesis):
    reference_tokens = str(reference).split()
    hypothesis_tokens = str(hypothesis).split()
    if not reference_tokens or not hypothesis_tokens:
        return 0.0
    return sentence_bleu([reference_tokens], hypothesis_tokens, weights=(1, 0, 0, 0), smoothing_function=smooth_fn)

combined_df['bleu1_score'] = combined_df.apply(
    lambda row: calculate_bleu1(row['answer_text'], row['generated_answer']), axis=1
)

# --------------------------
# Print overall averages
# --------------------------
print("Overall average F1 score:", combined_df['f1_score'].mean())

print("\nOverall average ROUGE scores:")
for col in rouge_df.columns:
    print(f"  {col}: {combined_df[col].mean()}")

print("\nOverall METEOR score:", combined_df['meteor_score'].mean())
print("Overall BLEU-1 score:", combined_df['bleu1_score'].mean())

# --------------------------
# Save results
# --------------------------
combined_df.to_csv("/content/all_languages_with_scores.csv", index=False)
print("\nSaved results to /content/all_languages_with_scores.csv")

Overall average F1 score: 0.31243927702179386

Overall average ROUGE scores:
  rouge1: 0.13871833125808944
  rouge2: 0.0608876560332871
  rougeL: 0.13871833125808944
  rougeLsum: 0.13871833125808944

Overall METEOR score: 0.41163174037000255
Overall BLEU-1 score: 0.23083037834157363

Saved results to /content/all_languages_with_scores.csv
